In [84]:
# h2o.shutdown()

In [85]:
import h2o
import numpy as np
import math
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning, module='.*/jupyter/.*')
h2o.init(nthreads = -1)




No instance found at ip and port: localhost:54321. Trying to start local jar...


JVM stdout: /var/folders/k_/kpp1czqs3957vq2pr5qngck00000gn/T/tmpmi3xAu/h2o_laurend_started_from_python.out
JVM stderr: /var/folders/k_/kpp1czqs3957vq2pr5qngck00000gn/T/tmpH2Dof_/h2o_laurend_started_from_python.err
Using ice_root: /var/folders/k_/kpp1czqs3957vq2pr5qngck00000gn/T/tmp3oP_Ba


Java Version: java version "1.8.0_73"
Java(TM) SE Runtime Environment (build 1.8.0_73-b02)
Java HotSpot(TM) 64-Bit Server VM (build 25.73-b02, mixed mode)


Starting H2O JVM and connecting: ........... Connection successful!


H2O cluster uptime:,1 seconds 260 milliseconds
H2O cluster version:,3.8.3.3
H2O cluster name:,H2O_started_from_python_laurend_gcr878
H2O cluster total nodes:,1
H2O cluster total free memory:,3.56 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321
H2O Connection proxy:,None


# Summary Notes

* use gbm (consider all options, for comparison)
* AUC to set threshold
* Compare machine learning models to regular group by models
* remove the customerID since that is the same as a row index
* todo: fill in missing values for TotalCharges column
* todo: compare different stratified functions and parameter to see if they stratify in the same way *(i.e. split_frame(ratios=None, destination_frames=None, seed=None), stratified_kfold_column(n_folds=3, seed=-1), stratified_split(test_frac=0.2, seed=-1, and fold_assignment="Stratified" for the GBM estimator)*


In [86]:
# A small clean telecommunications sample dataset (https://www.ibm.com/communities/analytics/watson-analytics-blog/predictive-insights-in-the-telco-customer-churn-data-set/
telco_dataset = h2o.import_file("https://s3.amazonaws.com/h2o-smalldata/TelcoChurn.csv")


Parse Progress: [##################################################] 100%


In [87]:
# get a summary of the dataset
print telco_dataset.nacnt()
telco_dataset.describe()

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 0.0]
Rows:7,043 Cols:21

Chunk compression summary: 


chunk_type,chunk_name,count,count_percentage,size,size_percentage
CBS,Bits,7,33.3333343,6.5 KB,2.8568119
C1N,1-Byte Integers (w/o NAs),11,52.3809552,76.4 KB,33.5680753
C2S,2-Byte Fractions,1,4.7619049,13.8 KB,6.0809709
C4S,4-Byte Fractions,1,4.7619049,27.6 KB,12.1258937
CStr,String,1,4.7619049,103.2 KB,45.3682482



Frame distribution summary: 


,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
127.0.0.1:54321,227.6 KB,7043.0,1.0,21.0
mean,227.6 KB,7043.0,1.0,21.0
min,227.6 KB,7043.0,1.0,21.0
max,227.6 KB,7043.0,1.0,21.0
stddev,0 B,0.0,0.0,0.0
total,227.6 KB,7043.0,1.0,21.0


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
type,string,enum,int,enum,enum,int,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,real,real,enum
mins,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.25,18.8,0.0
mean,NaN,0.504756495811,0.162146812438,0.483032798523,0.299588243646,32.3711486582,0.903166264376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.592219224762,NaN,64.7616924606,2283.30044084,0.265369870794
maxs,NaN,1.0,1.0,1.0,1.0,72.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,3.0,118.75,8684.8,1.0
sigma,NaN,0.500012873647,0.36861160561,0.49974751072,0.45811016751,24.5594810231,0.295752231784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.491456924049,NaN,30.0900470977,2266.77136188,0.441561305122
zeros,0,3488,5901,3641,4933,11,682,3390,2421,3498,3088,3095,3473,2810,2785,3875,2872,1544,0,0,5174
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0
0,7590-VHVEG,Female,0.0,Yes,No,1.0,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0.0,No,No,34.0,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0.0,No,No,2.0,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


In [88]:
# what does each column tell you, make sure type is correct, and see if new features need to be created with strings
telco_dataset.columns

[u'customerID',
 u'gender',
 u'SeniorCitizen',
 u'Partner',
 u'Dependents',
 u'tenure',
 u'PhoneService',
 u'MultipleLines',
 u'InternetService',
 u'OnlineSecurity',
 u'OnlineBackup',
 u'DeviceProtection',
 u'TechSupport',
 u'StreamingTV',
 u'StreamingMovies',
 u'Contract',
 u'PaperlessBilling',
 u'PaymentMethod',
 u'MonthlyCharges',
 u'TotalCharges',
 u'Churn']

In [89]:
# check whether the customerID column is unique per row, if so use as an index and remove from the predictors
telco_dataset['customerID'].asfactor().unique().nrow


7043

In [90]:
# select all columns as predictors except the customerID (which is like an index) and the response column
features_list = list(telco_dataset.columns[1:-1])
response_name = 'Churn'

In [91]:
# specify the response column
response_col = telco_dataset[response_name]
# get a list of the categorical levels in your response column
print 'the response classes are:',response_col.levels()
print 'number of classes:', response_col.nlevels()
print ''
# check that the response column is already interpreted as a factor (i.e. enum/categorical)
print 'Is the response column a categorical:',response_col.isfactor()
# check that there are two levels in our response column:
response_col.nlevels()
print ''
# check for missing values in the training set and specifically the response column
print 'there are {0} missing values in the dataset'.format(telco_dataset.isna().sum())
print 'there are {0} missing values in the labels'.format(telco_dataset[response_col].isna().sum())
print ''

print 'check for class imbalace'
print '------------------------'
num_train_samples = telco_dataset.shape[0]  # Total number of training samples
print 'the dataset is not imbalanced, neither class is less then 10% of the whole (as shown below)'
telco_dataset[response_name].table()['Count']/num_train_samples



the response classes are: [['No', 'Yes']]
number of classes: [2]

Is the response column a categorical: [True]

there are 11.0 missing values in the dataset
there are 0.0 missing values in the labels

check for class imbalace
------------------------
the dataset is not imbalanced, neither class is less then 10% of the whole (as shown below)


Count
0.73463
0.26537


In [92]:
# create a new column that is the same as tenure but it replaces tenure = 0 with tenure = 1
# tenure = 0 corresponds to people who have be a customer less then a month but have still payed
# the monthly fee:
telco_dataset['new_tenure_col'] = (telco_dataset['tenure'] == 0).ifelse(1, telco_dataset['tenure'])

# impute TotalCharges from MonthlyCharges and tenure (i.e. TotalCharges = tenure * MonthlyCharges)
# if TotalCharges value is missing fill in, if not leave as is
telco_dataset['TotalCharges'] = (telco_dataset['TotalCharges'].isna() == 1).ifelse(
    (telco_dataset['MonthlyCharges']  * telco_dataset['new_tenure_col']), telco_dataset['TotalCharges'])

# remove new tenure col when done
telco_dataset= telco_dataset.drop('new_tenure_col')
# check that there are no more missing values
print telco_dataset['TotalCharges'].isna().sum()

0.0


In [93]:
# print out the count for each categorical feature level (this excludes monthly charges and total charges)
for column_name in telco_dataset.columns[1:-3]:
    print telco_dataset[str(column_name)].table()
    print '~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'
    

gender      Count
--------  -------
Female       3488
Male         3555

[2 rows x 2 columns]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  SeniorCitizen    Count
---------------  -------
              0     5901
              1     1142

[2 rows x 2 columns]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Partner      Count
---------  -------
No            3641
Yes           3402

[2 rows x 2 columns]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Dependents      Count
------------  -------
No               4933
Yes              2110

[2 rows x 2 columns]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  tenure    Count
--------  -------
       0       11
       1      613
       2      238
       3      200
       4      176
       5      133
       6      110
       7      131
       8      123
       9      119

[73 rows x 2 columns]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
PhoneService      Count
--------------  -------
No                  682
Yes                6361

[2 rows x 2 columns]
~~~~~~~~~~~~~~~~~~~~~~~~

In [94]:
# split your datasets and set seed so the split is the same each time the code is run
train, valid, test = telco_dataset.split_frame(ratios=[0.70,0.15], seed=1234)

In [95]:
# check that the split went as expected, print out the dimensions of each dataset
# and then print their sum
print train.shape
print valid.shape
print test.shape
print train.shape[0] + valid.shape[0] + test.shape[0]

(4945, 21)
(1040, 21)
(1058, 21)
7043


In [96]:
# run gbm estimator with the default parameters to establish a benchmark model 

# import GBM estimator with default parameters (set seed for reproducibility)
from h2o.estimators.gbm import H2OGradientBoostingEstimator
default_model = H2OGradientBoostingEstimator(distribution= 'bernoulli', seed = 1234)

# train the default model
default_model.train(x=features_list, y=response_name, training_frame=train)

# get the AUC for the training set
print 'train auc:', default_model.auc() 

# get the AUC for the validation set
default_perf_on_valid = default_model.model_performance(valid)
print 'validation auc:', default_perf_on_valid.auc()


gbm Model Build Progress: [##################################################] 100%
train auc: 0.901181015724
validation auc: 0.848169251566


In [97]:
# using parameters from https://github.com/h2oai/h2o-3/blob/master/h2o-docs/src/product/tutorials/gbm/gbmTuning.ipynb
# this model overfits less on the training set
model_0 = H2OGradientBoostingEstimator(distribution='bernoulli',
                                    ntrees=10000,
                                    max_depth=4,
                                    learn_rate=0.01,
                                    stopping_rounds = 5, stopping_tolerance = 1e-4, stopping_metric = "AUC",
                                    sample_rate = 0.8,
                                    col_sample_rate = 0.8,
                                    seed = 1234,
                                    score_tree_interval = 10)

model_0.train(x=features_list, 
              y=response_name, 
              training_frame=train,
              validation_frame = valid)

# get the AUC for the training set
print 'train auc:', model_0.auc() 

# get the AUC for the validation set
model_0_perf_on_valid = model_0.model_performance(valid)
print 'validation auc:', model_0_perf_on_valid.auc()


gbm Model Build Progress: [##################################################] 100%
train auc: 0.87052692852
validation auc: 0.848343677777


## Grid Search
First we want to know what value of max_depth to use because it has a big impact on the model training time and optimal values depend strongly on the dataset. We'll do a quick Cartesian grid search to get a rough idea of good candidate max_depth values. Each model in the grid search will use early stopping to tune the number of trees using the validation set AUC, as before.

In [98]:
# using grid to get the best max_depths
hyper_params = {'max_depth' : range(1,30,2)}
search_criteria = {'strategy': "Cartesian"}

# build gbm model with grid search parameters
from h2o.grid.grid_search import H2OGridSearch
gs_1 = H2OGridSearch(H2OGradientBoostingEstimator(distribution='bernoulli',
                                    ntrees=10000,
                                    learn_rate=0.05,
                                    # learn_rate_annealing = 0.99, 
                                    sample_rate = 0.8,
                                    col_sample_rate = 0.8,
                                    seed = 1234,
                                    score_tree_interval = 10,              
                                    stopping_rounds = 5,
                                    stopping_metric = "AUC",
                                    stopping_tolerance = 1e-4),
                                    hyper_params = hyper_params,
                                    grid_id = 'grid_determines_max_depth',
                                    search_criteria = search_criteria)

# train grid search
gs_1.train(x=features_list, 
           y=response_name, 
           training_frame=train,
           validation_frame = valid)

# get the grid search results to see which max_depth performed the best
print(gs_1)


gbm Grid Build Progress: [##################################################] 100%
     max_depth                           model_ids              logloss
0            1   grid_determines_max_depth_model_0   0.4013688355087074
1            3   grid_determines_max_depth_model_1  0.40458862926940764
2            5   grid_determines_max_depth_model_2   0.4093168924776222
3            7   grid_determines_max_depth_model_3  0.42410004205579466
4            9   grid_determines_max_depth_model_4  0.43719712051905313
5           13   grid_determines_max_depth_model_6  0.45662427444412673
6           11   grid_determines_max_depth_model_5   0.4579035014754397
7           25  grid_determines_max_depth_model_12  0.46288373841706143
8           17   grid_determines_max_depth_model_8  0.46803023143599237
9           27  grid_determines_max_depth_model_13   0.4721069886844467
10          19   grid_determines_max_depth_model_9   0.4726359129628112
11          29  grid_determines_max_depth_model_14  

In [99]:
# print out the auc for all models, sorted from best to worst
auc_table = gs_1.sort_by('auc(valid=True)',increasing=False)
print(auc_table)


Grid Search Results for H2OGradientBoostingEstimator: 


Model Id,Hyperparameters: [max_depth],auc(valid=True)
grid_determines_max_depth_model_0,[1],0.8526801
grid_determines_max_depth_model_1,[3],0.8493733
grid_determines_max_depth_model_2,[5],0.8447897
grid_determines_max_depth_model_3,[7],0.8367443
grid_determines_max_depth_model_4,[9],0.8318822
grid_determines_max_depth_model_12,[25],0.8268553
grid_determines_max_depth_model_5,[11],0.8263224
grid_determines_max_depth_model_11,[23],0.8261697
grid_determines_max_depth_model_6,[13],0.8258233
grid_determines_max_depth_model_14,[29],0.8251498


In [100]:
# find the range of the max_depth for the top five models
new_auc_table = auc_table[1:5]
max_depths_to_use = new_auc_table['Hyperparameters: [max_depth]']
print max_depths_to_use

# get the max depths as a list
new_maxmin_list = []
for element in max_depths_to_use:
    new_maxmin_list.append(element[0])
new_max = max(new_maxmin_list)
new_min = min(new_maxmin_list)

([3], [5], [7], [9])


Now that we know a good range for max_depth, we can tune all other parameters in more detail. Since we don't know what combinations of hyper-parameters will result in the best model, we'll use random hyper-parameter search to "let the machine get luckier than a best guess of any human".

In [101]:
hyper_params_2 = {'max_depth' : list(range(new_min,new_max+1,1)),
                'sample_rate': [x/100. for x in range(20,101)],
                'col_sample_rate' : [x/100. for x in range(20,101)],
                'col_sample_rate_per_tree': [x/100. for x in range(20,101)],
                'col_sample_rate_change_per_level': [x/100. for x in range(90,111)],
                'min_rows': [2**x for x in range(0,int(math.log(train.nrow,2)-1)+1)],
                'nbins': [2**x for x in range(4,11)],
                'nbins_cats': [2**x for x in range(4,13)],
                'min_split_improvement': [0,1e-8,1e-6,1e-4],
                'histogram_type': ["UniformAdaptive","QuantilesGlobal","RoundRobin"]}
search_criteria_2 = {'strategy': "RandomDiscrete",
                   'max_runtime_secs': 3600,  ## limit the runtime to 60 minutes
                   'max_models': 100,  ## build no more than 100 models
                   'seed' : 1234,
                   'stopping_rounds' : 5,
                   'stopping_metric' : "AUC",
                   'stopping_tolerance': 1e-3
                   }

#Build grid search with GBM and hyper parameters
gs_2 = H2OGridSearch(H2OGradientBoostingEstimator(distribution='bernoulli',
                                    ntrees=10000,
                                    learn_rate=0.05,
                                    #learn_rate_annealing = 0.99,
                                    stopping_rounds = 5,
                                    stopping_tolerance = 1e-4,
                                    stopping_metric = "AUC", 
                                    score_tree_interval = 10,
                                    seed = 1234),
                                    hyper_params = hyper_params_2,
                                    grid_id = 'grid_2',
                                    search_criteria = search_criteria_2)

# train the grid and print results
gs_2.train(x=features_list, 
           y=response_name, 
           training_frame=train,
           validation_frame = valid)
print(gs_2)


gbm Grid Build Progress: [##################################################] 100%
      col_sample_rate col_sample_rate_change_per_level  \
0                0.27                             1.07   
1                 0.2                             0.96   
2                0.38                             1.07   
3                0.22                             1.06   
4                0.22                             1.09   
5                0.37                             0.94   
6                0.69                             0.97   
7                 0.4                             0.91   
8                0.71                              0.9   
9                0.74                             0.96   
10               0.92                             1.06   
11               0.25                             1.06   
12               0.99                              1.0   
13                0.7                             0.92   
14               0.67                         

In [65]:
# print out the auc for all of the models on the validation set
auc_table_2 = gs_2.sort_by('auc(valid=True)',increasing=False)
print(auc_table_2)


Grid Search Results for H2OGradientBoostingEstimator: 


Model Id,"Hyperparameters: [nbins, col_sample_rate, min_split_improvement, col_sample_rate_per_tree, min_rows, col_sample_rate_change_per_level, nbins_cats, sample_rate, histogram_type, max_depth]",auc(valid=True)
grid_2_model_0,"[512, 0.5, 1e-08, 0.65, 1024.0, 1.02, 512, 0.64, u'RoundRobin', 3]",0.8517493
grid_2_model_1,"[16, 0.27, 1e-06, 0.39, 128.0, 1.07, 64, 0.57, u'RoundRobin', 3]",0.8512867
grid_2_model_15,"[64, 0.4, 0.0, 0.34, 1024.0, 0.91, 2048, 0.48, u'UniformAdaptive', 5]",0.8508978
grid_2_model_8,"[16, 0.57, 0.0, 0.68, 4.0, 1.1, 4096, 0.58, u'RoundRobin', 1]",0.8507440
grid_2_model_3,"[256, 0.5, 1e-06, 0.81, 1024.0, 0.99, 4096, 0.76, u'QuantilesGlobal', 3]",0.8506612
grid_2_model_14,"[128, 0.5, 0.0, 0.92, 2.0, 0.94, 2048, 0.61, u'RoundRobin', 1]",0.8506376
grid_2_model_9,"[256, 0.44, 1e-06, 0.38, 16.0, 1.03, 512, 0.55, u'QuantilesGlobal', 2]",0.8504024
grid_2_model_2,"[128, 0.47, 1e-08, 0.68, 64.0, 0.98, 128, 0.87, u'UniformAdaptive', 4]",0.8501382
grid_2_model_13,"[512, 0.32, 1e-08, 0.88, 512.0, 1.02, 1024, 0.46, u'UniformAdaptive', 4]",0.8501054
grid_2_model_7,"[1024, 0.75, 1e-06, 0.8, 32.0, 0.99, 512, 0.5, u'UniformAdaptive', 3]",0.8500305


In [102]:
# print out the auc for all of the models on the validation set
auc_table_2 = gs_2.sort_by('auc(valid=True)',increasing=False)
print(auc_table_2)



Grid Search Results for H2OGradientBoostingEstimator: 


Model Id,"Hyperparameters: [nbins, col_sample_rate, min_split_improvement, col_sample_rate_per_tree, min_rows, col_sample_rate_change_per_level, nbins_cats, sample_rate, histogram_type, max_depth]",auc(valid=True)
grid_2_model_63,"[512, 0.37, 1e-06, 0.23, 16.0, 0.94, 4096, 0.47, u'QuantilesGlobal', 7]",0.8533899
grid_2_model_1,"[16, 0.27, 1e-06, 0.39, 128.0, 1.07, 64, 0.57, u'RoundRobin', 3]",0.8533827
grid_2_model_80,"[32, 0.38, 0.0, 0.35, 512.0, 1.07, 128, 0.28, u'QuantilesGlobal', 9]",0.8532276
grid_2_model_15,"[64, 0.4, 0.0, 0.34, 1024.0, 0.91, 2048, 0.48, u'UniformAdaptive', 7]",0.8532082
grid_2_model_73,"[32, 0.22, 0.0001, 0.55, 512.0, 1.06, 16, 0.28, u'RoundRobin', 8]",0.8531889
---,---,---
grid_2_model_24,"[512, 0.87, 1e-06, 0.7, 8.0, 0.9, 4096, 0.55, u'UniformAdaptive', 8]",0.8364173
grid_2_model_82,"[64, 0.44, 1e-06, 0.76, 8.0, 1.06, 256, 0.99, u'QuantilesGlobal', 8]",0.8362865
grid_2_model_48,"[256, 0.92, 0.0, 1.0, 1.0, 1.04, 16, 0.57, u'QuantilesGlobal', 9]",0.8355161
grid_2_model_4,"[64, 0.91, 1e-06, 0.97, 16.0, 1.02, 2048, 0.26, u'UniformAdaptive', 7]",0.8345204



See the whole table with table.as_data_frame()



In [103]:
# get the best model from the list (the model name listed at the top of the table)
best_model = h2o.get_model('grid_2_model_0')
test_performance_model = best_model.model_performance(test)

# get the performance on the test model
print test_performance_model.auc()

0.860041773779


In [104]:
# save a csv of the predictions:
# first get the predictions from the best model
best_model_predictions = best_model.predict(test)
best_model_predictions.head()


gbm prediction Progress: [##################################################] 100%


predict,No,Yes
No,0.97037,0.0296304
No,0.97855,0.0214502
Yes,0.618732,0.381268
No,0.824716,0.175284
Yes,0.523088,0.476912
Yes,0.341982,0.658018
Yes,0.460539,0.539461
Yes,0.687619,0.312381
No,0.978541,0.0214587
Yes,0.532255,0.467745


In [32]:
# export predictions as a csv to the data analysis directory
# h2o.h2o.export_file(best_model_predictions, force=True)

In [1]:
# download a POJO of the best model (best_model)
# h2o.download_pojo(best_model, force=True)